#  Teste do jogo Card Boxing

In [1]:
# Esta vai ser a classe-mãe das outras classes de efeito
class Effect:
    def __init__(self, param):
        self.param = param

    # Método para ser aplicado antes da defesa ser contada em cálculo. Ele modifica o dano base com multiplicadores (*2, *3, etc)
    def apply_pre_damage_modifier(self, base_damage):
        return base_damage

    # Método para gerenciar os efeitos posteriores ao dano. Descartar cartas, perder slots, novos efeitos em geral.
    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        pass

    # Método para debug
    def __repr__(self):
        return f"{self.__class__.__name__}({self.param})"
        
class DoubleDamageEffect(Effect):
    def __init__(self):
        self.multiplier = 2

    # Dobra o dano base
    def apply_pre_damage_modifier(self, base_damage, attacker, target, judge):
        # Checando se uma carta do tipo clinch foi utilizada
        if judge.has_used_clinch(target):
            doubled_damage = base_damage * self.multiplier
            judge.log_message(f"{attacker.name} attack caused Double Damage on {target.name} 'clinch'. {base_damage} double to {doubled_damage}")
            return doubled_damage
        else:
            judge.log_message(f"Double Damage condition failed. Target {target.name} did not use clinch.")
            return base_damage
        
class ClinchEffect(Effect):

    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        judge.log_message(f"{attacker.name} clinched {target.name}! Draw block for next turn activated.")

        # Juiz bloqueia a compra
        judge.set_draw_lock(target, self.param)

class InvincibleEffect(Effect):

    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        judge.log_message(f"{attacker.name} cannot be damaged or clinched.")

        judge.set_invulnerability(attacker)

class StopHittingYourselfEffect(Effect):
    def __init__(self):
        self.returned_damage = 0.20

    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        attacker_card = judge.current_turn_actions.get(attacker)
        is_attacker_attack = attacker_card and attacker_card.get("class") == "attack"

        if judge.check_card_class(target, "guard") and final_damage > 0 and is_attacker_attack:

            damage_reflected = int(final_damage * self.param)
            judge.log_message(f"{target.name} successfully defended and reflected {self.returned_damage*100}% damage!")
            judge.apply_damage(attacker, damage_reflected)

        elif not is_attacker_attack:
            judge.log_message(f"Effect failed. {attacker.name} card was not in the 'attack' class.")

class WhiteHotFirePunchEffect(Effect):
    def __init__(self):
        self.lost_slots = 1

    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        judge.log_message(f"{attacker.name}'s attack burned {target.name}'s hand and now it lost {self.lost_slots} slot.")

        judge.drop_hand_slot(target, self.lost_slots)

class OmniclinchEffect(Effect):
    def __init__(self):
        self.draw_lock = 1

    def apply_post_damage_effect(self, final_damage, attacker, target, judge, calculator):
        if target.is_invincible:
            judge.log_message(f"{target.name} in Invincible state, clinch blocked!")
            return

        judge.log_message(f"{attacker.name} activated Omniclinch. Draw block activated.")
        judge.set_draw_lock(target, self.draw_lock)

## static.py

In [2]:
from effects import *

# Tudo o que é estático (cartas, partes, etc) vai ficar armazenado aqui.
card_list = [
    
    # Criando as cartas de ação básicas.
    {
        "id": 1,
        "category": "card",
        "name": "Simple Guard", 
        "class": "guard", 
        "type": "neutral", 
        "description": "Simple Guard. Protects against Attack from opponent.It still can be Clinched",
        "effect": None
    },
    {   
        "id": 2,
        "category": "card",
        "name": "Simple Attack", 
        "class":"attack", 
        "type":"neutral", 
        "description": "Simple Attack. Does damage even if the opponent Attacks or Clinches. It can be blocked by Simple Guard.",
        "effect": None
    },
    {
        "id": 3,
        "category": "card",
        "name": "Clinch", 
        "class": "clinch", 
        "type": "neutral", 
        "description": "Clinch, holds the opponent fighter. Vulnerable to opponents attacks. Strong against opponents guards.",
        "effect": ClinchEffect
    },

        # Criando as cartas de ação especializadas
    {
        "id": 4,
        "category": "card",
        "name": "Strong Attack", 
        "class": "attack", 
        "type": "neutral", 
        "description": "Strong Attack. If the opponent clinches, it does double damage. Does regular damage against Simple Guard.",
        "effect": DoubleDamageEffect
    },
    {
        "id": 5, 
        "category": "card",
        "name": "Special Guard", 
        "class": "guard", 
        "type": "neutral", 
        "description": "Special Guard. Blocks completely any attack. Invulnerable to clinches.",
        "effect": InvincibleEffect
    }
]

# Lista de partes disponíveis para escolha
parts_list = [
    {
        "id": 1,
        "category": "part",
        "part_name": "Iron Head", 
        "slot": "head",
        "type": "iron",
        "description": "A solid Iron head made from pieces found in the scrapyard. If the fighter gets a full set of iron parts, it gets an extra bonus.",
        "modifiers": {
            "constitution": 2, 
            "strength": 0,
            "agility": -2,
            "HP": 2 
            }, 
            "special_card": {
                "id": 6, 
                "category": "card", 
                "name": "Iron Guard",
                "class": "guard", 
                "type": "iron", 
                "description": "The Iron Guard is so strong that it returns some damage to the oponent if they choose to attack.",
                "effect": StopHittingYourselfEffect
            },
        "resistances": ["water", "neutral"],
        "weaknesses": ["fire"]
    },
    {
        "id": 2,
        "category": "part",
        "part_name": "Blazing Arm", 
        "slot": "right_arm", 
        "type": "fire",
        "description": "A regular fighter arm bathed in a flammable substance and lit on fire. It increases the damage output but the fire also harms the user.",
        "modifiers": {
            "constitution": -2, 
            "strength": 2,
            "agility": 2,
            "HP": -2 
            }, 
            "special_card": {
                "id": 7, 
                "category": "card",
                "name": "Fiery Punch",
                "class": "attack", 
                "type": "fire", 
                "description": "Attack that leaves the opponent with a burn. If it lands, the opponent loses one slot of his hand for the end of the round.",
                "effect": WhiteHotFirePunchEffect
            },
        "resistances": [],
        "weaknesses": ["water"]
    },
    {
        "id": 3,
        "category": "part",
        "part_name": "Rubber Arm", 
        "slot": "left_arm",
        "type": "rubber",
        "description": "A rubber arm that is very flexible and helps holding the opponent. But it is not that powerful.",
        "modifiers": {
            "constitution": 1, 
            "strength": -1,
            "agility": 3,
            "HP": 0 
            }, 
            "special_card": {
                "id": 8, 
                "category": "card",
                "name": "Rubber Attack",
                "class": "attack", 
                "type": "rubber", 
                "description": "Hits weakly and automatically clinches if it lands.",
                "effect": OmniclinchEffect
            },
        "resistances": ["iron"],
        "weaknesses": ["fire"]
    },
    {
        "id": 4,
        "category": "part",
        "part_name": "Iron Body", 
        "slot": "body",
        "type": "iron",
        "description": "A body part made of iron. Very heavy and durable. If the fighter gets a full set of iron parts, it gets an extra bonus.",
        "modifiers": {
            "constitution": 6, 
            "strength": 0,
            "agility": -6,
            "HP": 4 
            },            
            "special_card": {
                "id": None, 
                "category": None,
                "name": None,
                "class": None, 
                "type": None, 
                "description": None,
                "effect": None
            },
        "resistances": ["water", "neutral"],
        "weaknesses": ["fire"]
    },
    {
        "id": 5,
        "category": "part",
        "part_name": "Rubber Body", 
        "slot": "body", 
        "type": "rubber",
        "description": "A body made of very flexible rubber. It is very effective withstanding attacks without losing agility. If the fighter gets a full set of rubber parts, it unlocks a new special card.",
        "modifiers": {
            "constitution": 2, 
            "strength": -1,
            "agility": 2,
            "HP": 0 
            },            
            "special_card": {
                "id": None, 
                "category": None,
                "name": None,
                "class": None, 
                "type": None, 
                "description": None,
                "effect": None,
            },
        "resistances": ["iron"],
        "weaknesses": ["fire"]
    },
    {
        "id": 6,
        "category": "part",
        "part_name": "Iron Leg", 
        "slot": "right_leg", 
        "type": "iron",
        "description": "Looks like a very thick nail, but who can tell? If the fighter gets a full set of iron parts, it gets an extra bonus.",
        "modifiers": {
            "constitution": 3, 
            "strength": 0,
            "agility": -2,
            "HP": 2 
            },            
            "special_card": {
                "id": None, 
                "category": None,
                "name": None,
                "class": None, 
                "type": None, 
                "description": None,
                "effect": None,
            },
        "resistances": ["water", "neutral"],
        "weaknesses": ["fire"]
    },
    {
        "id": 7,
        "category": "part",
        "part_name": "Rubber Leg", 
        "slot": "left_leg", 
        "type": "rubber",
        "description": "A leg made of very flexible rubber. It increases agility but sacrifices power. If the fighter gets a full set of rubber parts, it unlocks a new special card.",
        "modifiers": {
            "constitution": 1, 
            "strength": -2,
            "agility": 3,
            "HP": 0 
            },            
            "special_card": {
                "id": None, 
                "category": None,
                "name": None,
                "class": None, 
                "type": None, 
                "description": None,
                "effect": None,
            },
        "resistances": ["iron"],
        "weaknesses": ["fire"]
    }
]

# Matriz de conflitos
conflicts_table = {
    ('attack', 'guard'): {'winner':'p2','winning_class':'guard'}, 
    ('guard', 'attack'): {'winner':'p1','winning_class':'guard'}, 
    ('attack', 'clinch'): {'winner':'p1','winning_class':'attack'},
    ('clinch', 'attack'): {'winner':'p2','winning_class':'attack'},
    ('attack', 'attack'): {'winner':'tie','winning_class':'attack'},
    ('clinch', 'guard'): {'winner':'p1','winning_class':'clinch'},
    ('guard', 'clinch'): {'winner':'p2','winning_class':'clinch'},
    ('clinch', 'clinch'): {'winner':'tie','winning_class':'clinch'},
    ('guard', 'guard'): {'winner':'tie','winning_class':'none'},
}

# Matriz de resistências. Chave - Ataque, Valor - Resistência.
resistances_matrix = {
    "iron": ["water"],
    "rubber": ["fire"],
    "fire": ["water"],
    "neutral":["rubber", "iron"]
}

# Matriz de fraquezas. Chave - Ataque, Valor - Elemento com fraqueza aquele ataque.
weaknesses_matrix = {
    "iron": ["rubber"],
    "rubber": [],
    "fire": ["iron", "rubber"],
    "water": ["fire"],
    "neutral": []
}

## ui_manager.py

In [3]:
# Aqui fica um arquivo responsável por gerenciar a exibição das tabelas de cartas, peças e mensagens para o usuário

# Imports
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
from rich.layout import Layout

class UIManager:
    def __init__(self):
        self.console = Console()
        
    # Método para printar mensagens
    def printMessage(self, message:str):
        self.console.print(Text(message))

    # Método para criar a estrutura da tabela de cards
    @staticmethod
    def createCardsTable(title):
        # Cria a tabela
        cards_table = Table(title=title)
        
        #Cria as colunas da tabela
        cards_table.add_column("ID")
        cards_table.add_column("Card Name")        
        cards_table.add_column("Class")  
        cards_table.add_column("Type")
        cards_table.add_column("Description")

        return cards_table
    
    # Método para formatar cada linha da tabela
    def addCardRow(self, cards_table, card):
        
        cards_table.add_row(
            str(card["id"]),
            card["name"],
            card["class"],
            card["type"],
            card["description"]
#             ", ".join(part["resistances"]) if part["resistances"] else "N/A",
#             ", ".join(part["weaknesses"]) if part["weaknesses"] else "N/A"
        )


    # Método para criar a estrutura da tabela de status atuais
    @staticmethod
    def createCurrentStatsTable(title):
        # Cria a tabela
        current_stats_table = Table(title=title)
                
        #Cria as colunas da tabela
        current_stats_table.add_column("HP")
        current_stats_table.add_column("Con")        
        current_stats_table.add_column("Str")  
        current_stats_table.add_column("Agi")
        current_stats_table.add_column("Def")
        current_stats_table.add_column("Atk")
        current_stats_table.add_column("Cli")    
        current_stats_table.add_column("Resistances")        
        current_stats_table.add_column("Weaknesses")

        return current_stats_table
    
    # Método para formatar cada linha da tabela
    def addCurrentStatsRow(self, stats_table, stats_list):
        
        # Monta a tabela
        stats_table.add_row(*stats_list)

        # Printa a tabela
        self.console.print(stats_table)

    # Método para criar a estrutura da tabela de partes
    @staticmethod
    def createPartsTable(title):
        # Cria a tabela
        parts_table = Table(title=title)
        
        #Cria as colunas da tabela
        parts_table.add_column("ID")
        parts_table.add_column("Part Name")        
        parts_table.add_column("Slot")  
        parts_table.add_column("Type")
        parts_table.add_column("Description")
        parts_table.add_column("Con")        
        parts_table.add_column("Str")        
        parts_table.add_column("Agi")
        parts_table.add_column("HP")           
        parts_table.add_column("Resistances")        
        parts_table.add_column("Weaknesses")
        
        return parts_table

    # Método para formatar cada linha da tabela
    def addPartRow(self, parts_table, part):
        
        parts_table.add_row(
            str(part["id"]),
            part["part_name"],
            part["slot"],
            part["type"],
            part["description"],
            str(part["modifiers"]["constitution"]),
            str(part["modifiers"]["strength"]),
            str(part["modifiers"]["agility"]),
            str(part["modifiers"]["HP"]),
            ", ".join(part["resistances"]) if part["resistances"] else "N/A",
            ", ".join(part["weaknesses"]) if part["weaknesses"] else "N/A"
        )

    # Método para criar a estrutura da tabela de stats
    @staticmethod
    def createStatsTable(title):
        # Cria a tabela
        stats_table = Table(title=title)
                
        #Cria as colunas da tabela
        stats_table.add_column("HP")
        stats_table.add_column("Con")        
        stats_table.add_column("Str")  
        stats_table.add_column("Agi")
        stats_table.add_column("Def")
        stats_table.add_column("Atk")
        stats_table.add_column("Cli")    
        stats_table.add_column("Resistances")        
        stats_table.add_column("Weaknesses")

        return stats_table
    
    # Método para formatar cada linha da tabela
    def addStatsRow(self, stats_table, stats_list):
        
        # Monta a tabela
        stats_table.add_row(*stats_list)

        # Printa a tabela
        self.console.print(stats_table)
        
    @staticmethod
    def createSlotsTable(title):
        # Cria a tabela
        robot_slots = Table(title=title)

        # Cria a coluna da tabela
        robot_slots.add_column("Slot")
        robot_slots.add_column("Part ID")
        robot_slots.add_column("Part Name")
        robot_slots.add_column("Part Type")
        
        return robot_slots
    
    # Método para formatar cada linha da tabela
    def addSlotsRow(self, slots_table, slots_list):
        
        # Monta a tabela
        slots_table.add_row(*slots_list)

## robot.py

In [4]:
# Card Boxing Game

# Imports
# Imports rich são para melhorar a qualidade do display dos dados para o usuário
from rich.console import Console
from rich.table import Table
from static import parts_list
from ui_manager import UIManager

# Dois robôs boxeadores se enfrentam no ringue. Cada robô realiza uma ação que é declarada por meio de cartas.
# O jogo dura 3 rounds.
# Cada round acontece em 3 turnos.
# Cada turno é composto por uma ação de cada jogador e por suas consequências.

# Criando primeiro a classe robô.
class Robot:
    # Aqui o jogador define se ele quer um arquétipo de robô focado em ataque, defesa ou balanceado. Isso define os atributos base do robô.
    def __init__(self, archetype, ui_manager, robot_name=None):
        
        self.archetype = archetype
        
        if archetype == "atk":
            self.constitution = 6
            self.strength = 10
            self.agility = 8
            self.HP = 6
            
        elif archetype == "def":
            self.constitution = 10
            self.strength = 6
            self.agility = 6
            self.HP = 8
            
        elif archetype == "bal":
            self.constitution = 7
            self.strength = 7
            self.agility = 7
            self.HP = 7
            
        self.robot_name = robot_name
            
        self.defense = self.constitution + (0.6 * self.strength)
        self.attack = self.strength + (0.6 * self.agility)
        self.clinch = self.agility + (0.6 * self.strength)

        # Calcular o quanto de vida o jogador vai recuperar quando se levantar
        self.recover_modifier = 0

        self.resistances = []
        self.weaknesses = []
        
        # Dicionário com os slots do robô e suas partes
        self.slots = {
            "head": None,
            "body": None,
            "right_arm": None,
            "left_arm": None,
            "right_leg": None,
            "left_leg": None
        }

        self.ui = ui_manager
        
    # Método para mostrar os atributos e status do robô.
    def showStats(self):
        stats_table = self.ui.createStatsTable(f"Robot {self.robot_name}'s Stats")
        
        # Escrevendo a tabela
        stats_list = [
            str(self.constitution),
            str(self.strength),
            str(self.agility),
            str(self.HP),
            str(self.defense),
            str(self.attack),
            str(self.clinch),
            ", ".join(self.resistances) if self.resistances else "N/A",
            ", ".join(self.weaknesses) if self.weaknesses else "N/A"
            ]
        
        self.ui.addStatsRow(stats_table, stats_list)

    # Método para receber a peça de um slot.
    def getPartFromSlot(self, slot):
        return self.slots.get(slot, None)
        
    # Método para checar se o slot está vago.
    def checkSlotIfEmpty(self, slot):
        
        found_part = self.slots.get(slot)

        # Se não encontrar uma parte, retorna False
        if found_part is None:
            self.ui.printMessage(f"Slot '{slot}' is Empty.")
            return False
        # Se encontrar uma parte, retorna a parte.
        else:
            self.showParts(found_part["id"])
            return found_part
        
    # Método para exibir os slots e retornar a peça 
    def showSlots(self):
        
        robot_slots = self.ui.createSlotsTable(title=f"Robot {self.robot_name} Slots")
        
        # Iterando sobre as partes (head, body, etc)
        for slot_name in self.slots:
            # Retornando o dicionário da peça procurada, ou None caso não tenha peça
            part_data = self.getPartFromSlot(slot_name)
            
            if part_data:
                part_id = str(part_data["id"])
                part_name = part_data["part_name"]
                part_type = part_data["type"]
            
            else:
                part_id = "N/A"
                part_name = "N/A"
                part_type = "N/A"
                
            # Cria a lista com as informações
            robot_slots_list = [
                slot_name,
                part_id,
                part_name,
                part_type
            ]
            # Adiciona a lista na tabela
            self.ui.addSlotsRow(robot_slots, robot_slots_list)
        
        # Printa a tabela
        self.ui.console.print(robot_slots)
    
    # Colocando uma peça em um slot.
    def setSlot(self, slot, part):
        # A parte é referente ao slot correto?
        if part["slot"] == slot:
            # Checando se o slot está vazio
            if self.checkSlotIfEmpty(slot) is False:
                
                self.ui.printMessage(f"Slot {slot} empty and available. Printing current robot stats.")
                
                self.showStats()
                
                # Adicionando os modificadores nos atributos base do robô
                self.slots[part["slot"]] = part
                self.constitution += part["modifiers"]["constitution"]
                self.strength += part["modifiers"]["strength"]
                self.agility += part["modifiers"]["agility"]
                self.HP += part["modifiers"]["HP"]

                for i in part["resistances"]:
                    self.resistances.append(i)
                for i in part["weaknesses"]:
                    self.weaknesses.append(i)
                
                self.ui.printMessage(f"Part {part['part_name']} equipped in slot {slot}")
                self.ui.printMessage(f"New stats after part was equipped:")
                self.showStats()
                
                return self.showSlots()
            
            # Se o slot estiver ocupado
            else:
                # Removendo os modificadores antigos
                self.ui.printMessage(f"Slot {slot} already equipped with {self.slots[slot]['part_name']} part. Showing current stats.")
                self.showStats()
                
                self.ui.printMessage("Removing currently equipped part.")
                
                old_part = self.slots[slot]
                self.constitution -= old_part["modifiers"]["constitution"]
                self.strength -= old_part["modifiers"]["strength"]
                self.agility -= old_part["modifiers"]["agility"]
                self.HP -= old_part["modifiers"]["HP"]
                
                # Removendo as resistências e fraquezas da parte antiga.
                for i in old_part["resistances"]:
                    self.resistances.remove(i)
                for i in old_part["weaknesses"]:
                    self.weaknesses.remove(i)
                
                self.ui.printMessage(f"Part {old_part['part_name']} removed.")
                
                # Adicionando os modificadores nos atributos base do robô
                self.slots[part["slot"]] = part
                self.constitution += part["modifiers"]["constitution"]
                self.strength += part["modifiers"]["strength"]
                self.agility += part["modifiers"]["agility"]
                self.HP += part["modifiers"]["HP"]

                for i in part["resistances"]:
                    self.resistances.append(i)
                for i in part["weaknesses"]:
                    self.weaknesses.append(i)
                
                self.ui.printMessage(f"Part {part['part_name']} equipped in slot {slot}")
                self.ui.printMessage(f"New stats after part was equipped:")
                self.showStats()
                
                return self.showSlots()
       
        else:
            self.ui.printMessage(f"The part {part['part_name']} is meant to be equipped in the {part['slot']} slot. Not in the {slot} slot.")
    
    # Removendo uma peça de um slot.
    def cleanSlot(self, slot):
        
        # Checando se o slot está vazio
        if self.checkSlotIfEmpty(slot) is False:

            print(f"Slot {slot} already empty and available. Printing current robot stats.")

            self.showStats()

            return self.showSlots()

        # Se o slot estiver ocupado
        else:
            # Removendo os modificadores antigos
            print(f"Slot {slot} equipped with {self.slots[slot]['part_name']} part. Showing current stats.")
            self.showStats()

            print("Removing currently equipped part.")

            old_part = self.slots[slot]
            self.constitution -= old_part["modifiers"]["constitution"]
            self.strength -= old_part["modifiers"]["strength"]
            self.agility -= old_part["modifiers"]["agility"]
            self.HP -= old_part["modifiers"]["HP"]

            # Removendo as resistências e fraquezas da parte antiga.
            for i in old_part["resistances"]:
                self.resistances.remove(i)
            for i in old_part["weaknesses"]:
                self.weaknesses.remove(i)
                
            self.slots[slot] = None

            print(f"Part {old_part['part_name']} removed.")

            print(f"Updated stats after part was removed:")
            self.showStats()

            return self.showSlots()

    # Mostrar partes disponíveis
    def showParts(self, id_filter=None, slot_filter=None):
        
        # Primeiro, definindo o título
        if  id_filter is not None:
            title = f"Part Details (ID: {id_filter})"
        elif slot_filter is not None:
            title = f"Parts for Slot: {slot_filter}"
        else:
            title = f"All Parts"
        
        # Criando a tabela
        parts_table = self.ui.createPartsTable(title)    

        # Filtragem das partes
        found_parts = False
        
        for part in parts_list:
            
            # Manda continuar apenas se o id fornecido for diferente do id da parte iterada.
            if id_filter is not None and part["id"] != id_filter:
                continue
            
            # Mesma dinâmica, porém para as peças com slot diferente do procurado
            if slot_filter is not None and part["slot"] != slot_filter:
                continue
                
            # Caso isso tenha sido passado por ambos os filtros. Criamos a variável
            found_parts = True

            self.ui.addPartRow(parts_table, part)
            
            # Caso seja uma busca por ID único, a iteraçao acaba assim que encontra o ID
            if id_filter is not None:
                break
        
        # Valores de saída
        if found_parts:
            self.ui.console.print(parts_table)
        else:
            self.ui.printMessage("No pieces found.")


## deck.py

In [5]:
from static import card_list

# Método para encontrar uma carta pelo ID
def findCardById(card_list, card_id):
    for card_data in card_list:
        if card_data.get("id") == card_id:
            return card_data
    return None

# Criando a classe deck.
class Deck:
    def __init__(self, archetype):

        self.deck = []

        cards = card_list

        # Criando o deck básico do jogador com 3 cartas de cada tipo.
        for i in range(3):
            self.deck.append(
                findCardById(cards, 2)
                )
        for i in range(3):
            self.deck.append(
                findCardById(cards, 1)
                )
        for i in range(3):
            self.deck.append(
                findCardById(cards, 3)
                )
        
        # Colocando uma carta especial para cada arquétipo diferente
        if archetype == "atk":
            for i in range(2):
                self.deck.append(
                    findCardById(cards, 4)
                    )
        elif archetype == "def":
            for i in range(2):
                self.deck.append(
                    findCardById(cards, 5)
                    )
        elif archetype == "bal":
            self.deck.append(
                findCardById(cards, 4)
                )
            self.deck.append(
                findCardById(cards, 5)
                )

## player.py

In [6]:
# Imports
from robot import Robot
from deck import Deck
from rich.console import Console
from rich.table import Table
from ui_manager import UIManager
import random

# Criando a classe jogador
class Player:
    def __init__(self, name, robot_name, archetype, ui_manager: UIManager):
        self.name = name

        self.ui = ui_manager
        
        # Robô do jogador
        self.robot = Robot(archetype, self.ui, robot_name)

        # Deck base do jogador
        self.deck = Deck(archetype)

        # Cartas descartadas
        self.graveyard = []

        # Mão do jogador
        self.hand = []

        # Número máximo de cartas na mão
        self.max_hand_slots = 3
        # Atributo privado com o mínimo de uma carta na mão
        self._max_hand_slots: int = 1

        # Deck completo do jogador
        self.game_deck = []
        
        # Console para printar as cartas
        self.player_console = Console()

        # Atributos do jogador
        self.initial_constitution = self.robot.constitution
        self.initial_strength = self.robot.strength
        self.initial_agility = self.robot.agility
        self.initial_HP = self.robot.HP
        self.initial_defense = self.robot.defense
        self.initial_attack = self.robot.attack
        self.initial_clinch = self.robot.clinch

        # Contador de Quedas Totais
        self.fall_counter = 0

        # Contador de Quedas por Round - (Vai ser resetado com o round)
        self.falls = 0
        
        # Modificador de invencibilidade
        self.is_invincible = False
        
        # Modificador de compra de cartas
        self.draw_blocked = False

    # Permite ler o valor dos slots
    @property 
    def max_hand_slots(self) -> int:
        return self._max_hand_slots
    
    # Valida que o valor do atributo nunca seja menor do que 1
    @max_hand_slots.setter
    def max_hand_slots(self, value: int):
        
        if value < 1:
            self.ui.printMessage(f"Minimum slots for player {self.name}'s hand: 1 Slot")
            self._max_hand_slots = 1
        else:
            self._max_hand_slots = value
            
    # Adiciona as cartas especiais, caso existam
    def addSpecialCards(self):
        for part in self.robot.slots.values():
            if part is not None:
                
                special_card = part.get("special_card")
                
                if special_card and special_card.get("id") is not None:
                    self.deck.deck.append(special_card)

    # Cria o deck disponível para jogar, retorna uma lista
    def setDeck(self):
        
        # Adiciona as cartas especiais
        self.addSpecialCards()
        self.game_deck = self.deck.deck
        
        # Cria a tabela de cartas
        deck_table = self.ui.createCardsTable(f"Player {self.name}'s Deck")
        # Carrega a tabela
        for card in self.game_deck:
            self.ui.addCardRow(deck_table, card)
        
        self.ui.console.print(deck_table)
        # Retorna o game_deck já com todas as cartas inseridas
        return self.game_deck
    
    # Checa se o game deck tem n cartas possíveis para compra
    def checkGameDeck(self, quantity):
        # Retorna True se existe mais cartas para serem compradas no game deck do que o que foi pedido
        return len(self.game_deck) >= quantity
    
    # Embaralha o deck
    def shuffleDeck(self):
        return random.shuffle(self.game_deck)

    # Compra n cartas
    def getCard(self, num_of_cards):
        # Checagem se o game deck tem n cartas disponíveis
        if not self.checkGameDeck(num_of_cards):
            # Se não tiver, ele vai transformar num_of_cards na mesma quantidade de cards restantes.
            num_of_cards = len(self.game_deck)
            # Avisa quantas cartas estão disponíveis para compra no caso do game deck ser insuficiente.
            self.ui.printMessage(f"Only {num_of_cards} cards available. ")
        
        # Compra de cartas
        cards_drawn = 0
        while cards_drawn < num_of_cards:
            # Checa se a mão está cheia
            if len(self.hand) >= self.max_hand_slots:
                self.ui.printMessage(f"Player {self.name}'s hand is full!'")
                break
                
            # Checa se o deck ficou vazio durante a compra
            if not self.game_deck:
                break

            # Compra a carta do topo do baralho (índice 0)
            card = self.game_deck.pop(0)
            self.hand.append(card)
            
            cards_drawn += 1
        
        return True

    # Mostra a mão
    def showHand(self):
        hand_table = self.ui.createCardsTable(f"Player {self.name}'s Hand")
        
        for i in self.hand:
            self.ui.addCardRow(hand_table,i)
            
        self.ui.console.print(hand_table)

    # Mostra o cemitério
    def showGraveyard(self):
        graveyard_table = self.ui.createCardsTable(f"Player {self.name}'s Graveyard")
        
        for i in self.graveyard:
            self.ui.addCardRow(graveyard_table,i)
            
        self.ui.console.print(graveyard_table)
            
    # Escolhe a carta
    def chooseCard(self):
        self.showHand()
        chosen_card = int(input("Choose a card:"))
        return self.hand[chosen_card]
    
    def playCard(self):
        chosen_card = self.chooseCard()
        self.hand.remove(chosen_card)
        self.graveyard.append(chosen_card)
        return chosen_card
    
    # Mostra o status atual
    def showCurrentStatus(self):
        # Cria a tabela
        current_stats_table = self.ui.createCurrentStatsTable(f"{self.name}'s {self.robot.robot_name} Robot Stats")
        
        # Monta a tabela
        current_stats_list = [
            f"{self.initial_HP}/{self.robot.HP}",
            f"{self.initial_constitution}/{self.robot.constitution}",
            f"{self.initial_strength}/{self.robot.strength}",
            f"{self.initial_agility}/{self.robot.agility}",
            f"{self.initial_defense}/{self.robot.defense}",
            f"{self.initial_attack}/{self.robot.attack}",
            f"{self.initial_clinch}/{self.robot.clinch}"
        ]

        self.ui.addCurrentStatsRow(current_stats_table, current_stats_list)
        


## GameJudge.py (arquivo ainda não criado)

In [7]:
# Imports
from player import Player
from deck import Deck
from robot import Robot
from static import conflicts_table
from ui_manager import UIManager
from typing import Dict, Any, Tuple, List, TYPE_CHECKING
import random

# Criando uma classe "juiz" para conferir os status do deck e do robô de cada jogador.
class GameJudge:
    def __init__(self, ui_manager: UIManager, damage_calculator):
        # Inicializando o gerenciador de UI
        self.ui = ui_manager

        # Inicializando a calculadora de danos
        self.damage_calculator = damage_calculator

        # Dados temporários (renovados por turno)
        self.current_turn_actions = {}
        self.turn_damage_log = {}

        # Controladores de jogo
        self.turn = 0
        self.round = 0
        self.score = {}

    # Métodos auxiliares
        # Método para printar as mensagens para o jogo
    def log_message(self, message):
        self.ui.printMessage(f"Turn {self.turn} - {message}")
        
    # Métodos para registrar informações
        # Registrar carta lançada
    def register_action(self, player, card):
        # Registra a carta jogada
        self.current_turn_actions[player] = card

        # Mostra a mensagem
        self.log_message(f"Judge: {player.name} played '{card.get('name')}'.") 

        # Registrar o dano que vai ser aplicado
    def register_damage_dealt(self, player, damage):
        self.turn_damage_log[player] = damage

        # Limpa os registros do turno
    def clear_turn_records(self):
        self.turn_damage_log = {}
        self.current_turn_actions = {}

        # Aumentar 1 turno
    def increase_turn(self):
        self.turn += 1

        # Aumentar 1 round
    def increase_round(self):
        self.round += 1
        
        # Criar o score do jogo
    def create_score(self, player1, player2):
        self.score[player1] = 0
        self.score[player2] = 0
        
        # Aumentar 1 ponto para o jogador
    def give_point(self, player):
        self.score[player] += 1
        
        # Retornar o vencedor por score
    def return_score_winner(self):
        return max(self.score, key=self.score.get)

    # Métodos para checagem de ações
        # Conferir se clinch foi utilizado
    def has_used_clinch(self, player):
        return self.check_card_class(player, "clinch")

        # Conferir qual o tipo da carta utilizada
    def check_card_class(self, player, card_class) -> bool:
        if player in self.current_turn_actions:
            card = self.current_turn_actions[player]
            return card.get("class") == card_class
        return False
        
        # Conferir se a ação causou dano
    def check_successful_attack(self, player) -> bool:
        return any(damage > 0 for damage in self.turn_damage_log.values())

        # Checa se o jogador não pode comprar cartas no próximo turno
    def is_draw_locked(self, player) -> bool:
        return player.draw_blocked

    # Métodos que alteram estado
        # Método para definir status de invulnerável
    def set_invulnerability(self, player):
        player.is_invincible = True
        self.log_message(f"Judge: {player.name} invulnerability enabled!")
        
        # Método para remover status de invulnerável
    def reset_invulnerability(self, player):
        player.is_invincible = False
        self.log_message(f"Judge: {player.name} invulnerability disabled!")

        # Método para retornar True se estiver invulnerável
    def is_invulnerable(self, player):
        return player.is_invincible

        # Método para determinar quem ganha a disputa das cartas
    def determine_conflict(self, card_p1: Dict[str, Any], card_p2: Dict[str, Any]) -> Dict[str, Any]:
        
        # Obtém a classe da carta
        class_p1 = card_p1.get('class')
        class_p2 = card_p2.get('class')
        
        # Cria a chave de busca 
        key = (class_p1, class_p2)
        
        # Busca o resultado na tabela
        result = conflicts_table.get(key)
        
        return result

    # Método que retorna um dicionário com as informações do conflito antes de calcular dano e com base no resultado das cartas.    
    def return_conflict_result(self, player1, player2, card_p1, card_p2, result):

        winner = None
        loser = None

        if result['winner'] == 'p1':
            type = 'win'
            winner = player1
            loser = player2

        elif result['winner'] == 'p2':
            type = 'win'
            winner = player2
            loser = player1

        elif result['winner'] == 'tie':
            if result['winning_class'] == 'attack':
                type = 'attack_draw'
            elif result['winning_class'] == 'clinch':
                type = 'clinch_draw'
            elif result['winning_class'] == 'guard':
                type = 'guard_draw'

        result = {
            'type': type,

            'winner': winner,
            'loser': loser,

            'p1': {
                'player': player1,
                'card': card_p1,
                'class': card_p1['class'],
                'effect': card_p1['effect'],
                'speed': player1.initial_agility
            },

            'p2': {
                'player': player2,
                'card': card_p2,
                'class': card_p2['class'],
                'effect': card_p2['effect'],
                'speed': player2.initial_agility
            }
        }

        return result
    
    # Método para resolver o conflito de acordo com o dicionário passado como argumento
    def resolve_conflict_after_result(self, conflict_result):
        # Variáveis iniciais
        type = conflict_result['type']

        # Dicionário dos jogadores
        p1 = conflict_result['p1']
        p2 = conflict_result['p2']

        # Acesso aos objetos Player
        player1 = p1['player']
        player2 = p2['player']

        # No caso de não empate
        winner = conflict_result['winner']
        loser = conflict_result[ 'loser']

        # No caso de vitória
        if type == 'win':
            self.resolve_win(conflict_result)
        elif type == 'attack_draw':
            self.resolve_attack_draw(conflict_result)
        elif type == 'clinch_draw':
            self.resolve_clinch_draw(conflict_result)
        elif type == 'guard_draw':
            self.resolve_guard_draw(conflict_result)

    # Método que bloqueia a compra de cartas por 1 turno (efeito clinch)
    def set_draw_lock(self, player):
        player.draw_blocked = True
        
    # Método que bloqueia a compra de cartas por 1 turno (efeito clinch)
    def reset_draw_lock(self, player):
        player.draw_blocked = False

    #     # Método para aplicar dano
    # def apply_damage(self, player, damage):
    #     # Checa invulnerabilidade
    #     if player.is_invincible:
    #         self.log_message(f"Judge: {player.name} is invulnerable, no damage applied.")
    #         return True

    #     # Aplicando dano ao robô
    #     if damage > 0:
    #         player.initial_HP -= damage
    #         self.log_message(f"Judge: {player.name} received {damage} damage! HP left: {player.initial_HP}")

        # Método para dropar um slot da mão do jogador
    def drop_hand_slot(self, player, num_slots):
        current_limit = player.max_hand_slots
        new_limit = max(0, current_limit - num_slots)

        player.max_hand_slots = new_limit
        
        self.log_message(f"Judge: Cards in hand limit of the {player.name} player dropped {num_slots}. New limit: {player.max_hand_slots}.")
        
    # Método para checar a HP do robô.
    def is_knocked_out(self, player) -> bool:
        # Retorna True se a vida estiver menor ou igual a 0
        return player.initial_HP <= 0

    # Método para checar se o jogador tem cartas disponíveis no deck
    def is_deck_out(self, player) -> bool:
        # Retorna True se o jogador não tiver cartas jogáveis
        return len(player.game_deck) <= 0
    
    # Método para checar se o jogador tem cartas disponíveis na mão
    def is_hand_out(self, player) -> bool:
        # Retorna True se o jogador não tiver cartas jogáveis
        return len(player.hand) <= 0
    
    # Método para checar se o jogador está incapacitado
    def is_incapacitaded(self, player) -> bool:
        return self.is_deck_out(player) and self.is_hand_out(player)

    # Método para retornar o custo de cartas pagas por queda
    def apply_ko_cost(self, player):
        fall_counter = player.fall_counter

        # Calculando o custo que deve ser pago a cada queda 
        fall_cost = (fall_counter + 1) * 2
        player.fall_counter += 1

        # Retorna o custo que deve ser pago e modifica o contador de quedas do jogador.
        return int(fall_cost)

    # Método para aplicar KO
    def apply_ko(self, player, fall_cost: int) -> bool:
        # Retorna True se o número de cartas restantes no baralho for menor do que o custo.
        return len(player.game_deck) < fall_cost
    
    # Método para declarar vencedor
    def declare_winner(self, winner, reason):
        self.log_message("---GAME OVER---")
        self.log_message(f"Winner: {winner.name}")
        self.log_message(reason)
        
        raise SystemExit("Game Over, Thanks for playing!")
        
    # Método para verificar se o jogador caiu, e se ele consegue se levantar
    def player_down(self, attacker, target):
        if self.is_knocked_out(target):
            
            if self.is_incapacitaded(target):
                self.declare_winner(attacker, "K.O.")
            else:
                ko_cost = self.apply_ko_cost(target)
                if self.apply_ko(target, ko_cost):
                    self.declare_winner(attacker, "K.O.")
                else:
                    if not self.discard_from_deck(target, ko_cost):
                        self.declare_winner(attacker, "K.O.")
                    else: 
                        self.discard_from_deck(target, ko_cost)
        return

    # Método para resetar o contador de quedas
    def reset_player_falls(self, player):
        player.falls = 0

    # Método para somar uma queda ao contador
    def add_fall(self, player):
        player.falls +=1

    # Método para conferir se o jogador atingiu o número de quedas limite no round.
    def check_falls(self, player, num_falls) -> bool:
        # Retorna True se o jogador acumular 3 quedas no round.
        return player.falls == num_falls
    
    # Método para descartar tanto da mão quanto do deck
    def discard_from_deck(self, player, num_cards):
        total_cards = len(player.game_deck) + len(player.hand)

        # Se já não há cartas o suficiente para serem descartadas
        if num_cards >= total_cards:
            self.log_message(f"Player {player.name} does not have enough cards to discard.")
            return False
        
        # Descartando primeiro do deck
        cards_to_discard_from_deck = min(num_cards, len(player.game_deck))

        for card in range(cards_to_discard_from_deck):
            random_card = random.choice(player.game_deck)
            player.game_deck.remove(random_card)
            player.graveyard.append(random_card)

        # Descarte da mão se necessário
        remaining_to_discard = num_cards - cards_to_discard_from_deck

        cards_to_discard_from_hand = 0

        if remaining_to_discard > 0:
            cards_to_discard_from_hand = remaining_to_discard

            for card in range(cards_to_discard_from_hand):
                random_card = random.choice(player.hand)
                player.hand.remove(random_card)
                player.graveyard.append(random_card)
        self.log_message(f"Player {player.name} discarded {cards_to_discard_from_deck} cards from deck and {cards_to_discard_from_hand} cards from hand.")
        return True    
        
    # Método para retirar uma carta da mão do jogador
    def discard_from_hand(self, player, num_cards):
        if num_cards < len(player.hand):
            for card in range(num_cards):
                random_card = random.choice(player.hand)
                player.hand.remove(random_card)
                player.graveyard.append(random_card)
            self.log_message(f"Player {player.name} discarded {num_cards} cards.")
        else:
            for card in len(player.hand):
                random_card = random.choice(player.hand)
                player.hand.remove(random_card)
                player.graveyard.append(random_card)
            self.log_message(f"Player {player.name} discarded {num_cards} cards.")

    # Reseta o placar
    def reset_score(self, player):
        player.score = 0
    
    # Adiciona um ponto ao placar do jogador
    def add_point(self, player):
        player.score += 1

# Damage Calculator

In [51]:
from ui_manager import UIManager
from static import *
# Classe que vai ser a "calculadora" do juiz. Ela vai calcular, aplicar e retornar os modificadores e o dano.
class DamageCalculator:
    def __init__(self, ui_manager: UIManager):
        self.ui = ui_manager
    
    # Método para checar as fraquezas
    @staticmethod
    def check_weaknesses(card, defender):
        # Pega o tipo da carta atacante
        atk_type = card["type"]
        # Pega a lista de fraquezas do robô defensor
        def_weaknesses = defender.robot.weaknesses
        weaknesses_multiplier = 0
        for weakness in def_weaknesses:
            if weakness in weaknesses_matrix[atk_type]:
                weaknesses_multiplier += 0.5
        # Retorna o multiplicador de dano
        return weaknesses_multiplier
    
    # Método para checar as resistências
    @staticmethod
    def check_resistances(card, defender):
        # Pega o tipo da carta atacante
        atk_type = card["type"]
        # Pega a lista de resistências do robô defensor
        def_resistances = defender.robot.resistances
        resistances_multiplier = 0
        for resistance in def_resistances:
            if resistance in resistances_matrix[atk_type]:
                resistances_multiplier += 1
        # Retorna o multiplicador de dano
        return resistances_multiplier
    
    # O dano base é o dano a ser aplicado depois de considerar o ataque, fraquezas e resistências
    @staticmethod
    def get_base_damage(attacker, weaknesses_multiplier, resistances_multiplier):
        base_atk = attacker.initial_attack
        base_damage = base_atk * (1 + weaknesses_multiplier) * (0.5)**resistances_multiplier
        return base_damage

    # A defesa base leva em conta o atributo de defesa do robô. E quanto maior a defesa, menos eficientes os pontos de defesa adicionais se tornam.
    @staticmethod
    def get_base_defense(defender):
        base_def = defender.initial_defense
        base_defense = 100 / (100 + base_def)
        return base_defense

    # O dano final já leva em conta a defesa e retorna o número final que vai ser deduzido dos pontos de vida do defensor.
    @staticmethod
    def apply_mitigation(base_damage, base_defense):
        final_damage = base_damage * base_defense
        return final_damage

## turn.py (arquivo ainda não criado)

In [32]:
from effects import *
from player import Player
from game_judge import GameJudge
from damage_calculator import DamageCalculator
import random

# Criando a classe Game - Ela vai iniciar uma partida
class Turn:
    def __init__(self, player1, player2, gamejudge, damagecalculator):
        self.player1 = player1
        self.player2 = player2
        self.gamejudge = gamejudge
        self.damagecalculator = damagecalculator
        self.gamejudge.create_score(self.player1, self.player2)
    
# Estrutura do turno
    # Primeira fase - Resetando as variáveis
    def execute_first_phase(self):
        # Juiz limpa o log de atividades
        self.gamejudge.clear_turn_records()
        
        # Jogadores perdem invencibilidade
        self.gamejudge.reset_invulnerability(self.player1)
        self.gamejudge.reset_invulnerability(self.player2)
        
        # Se ambos não possuírem cartas em mãos e não possuírem cartas para comprar - O jogo acaba e o resultado vai para os pontos.
        p1_out = self.gamejudge.is_incapacitaded(self.player1)
        p2_out = self.gamejudge.is_incapacitaded(self.player2)
        
        if p1_out and p2_out:
            score = self.gamejudge.score
            for player, player_score in score:
                self.gamejudge.log_message(f"The {player.name} has scored {player_score} points.")
            winner = self.gamejudge.return_score_winner()
            self.gamejudge.declare_winner(winner, "Score")
   
            # Se nao possuir cartas em mãos E não possuir cartas para comprar - O jogador está derrotado.
        if p1_out and not p2_out:
            reason = f"{self.p1.name} is incapacitaded to fight!"
            self.gamejudge.declare_winner(self.player2, "Incapacitated Opponent.")        
        if p2_out and not p1_out:
            reason = f"{self.p2.name} is incapacitaded to fight!"
            self.gamejudge.declare_winner(self.player1, "Incapacitated Opponent.")
            
    
    # Segunda fase - Executando o começo do round        
    def execute_second_phase(self):        
        # Jogadores compram as cartas se forem permitidos.
        p1_draw_block = self.player1.draw_blocked
        p2_draw_block = self.player2.draw_blocked
        
        # Se o jogador 1 não estiver bloqueado
        if not p1_draw_block:
            # E se o jogador 1 tiver cartas na mão menos do que os slots da mão livres
            if len(self.player1.hand) < self.player1.max_hand_slots:
                self.player1.getCard(1)

        # Se o jogador 2 não estiver bloqueado
        if not p2_draw_block:
            # E se o jogador 1 tiver cartas na mão menos do que os slots da mão livres
            if len(self.player2.hand) < self.player2.max_hand_slots:
                self.player2.getCard(1)
                
        # Ambos os jogadores escolhem uma carta para jogar
        p1_card = self.player1.playCard()
        p2_card = self.player2.playCard()
        
        # Juiz registra ambas as cartas no turno
        self.gamejudge.register_action(self.player1, p1_card)
        self.gamejudge.register_action(self.player2, p2_card)
        
    # Terceira fase - Executando o conflito entre as cartas
    def execute_third_phase(self):
        # As cartas são colocadas em análise.

            # Criando as cartas que foram jogadas de acordo com o que o juiz registrou
        card_player1 = self.gamejudge.current_turn_actions[self.player1]
        card_player2 = self.gamejudge.current_turn_actions[self.player2]
        
        # Criando as variáveis que vão ser utilizadas e modificadas durante o turno.
            # Player 1
        p1_name = self.player1.name
        p1_weaknesses_multiplier = self.damagecalculator.check_weaknesses(card_player2, self.player1)
        p1_resistances_multiplier = self.damagecalculator.check_resistances(card_player2, self.player1)
        p1_base_defense = self.damagecalculator.get_base_defense(self.player1)
        p1_current_action = card_player1['class']
        p1_card_type = card_player1['type']

            # Player 2
        p2_name = self.player2.name
        p2_weaknesses_multiplier = self.damagecalculator.check_weaknesses(card_player1, self.player2)
        p2_resistances_multiplier = self.damagecalculator.check_resistances(card_player1, self.player2)
        p2_base_defense = self.damagecalculator.get_base_defense(self.player2)
        p2_current_action = card_player2['class']
        p2_card_type = card_player2['type']
            
        # Resolução dos conflitos das cartas (Jan Ken Pon)
        conflict_result = self.gamejudge.determine_conflict(card_player1, card_player2)

        # Se a carta for Special Guard, pula a parte de dano porque nada vai ser aplicado.
        if card_player1['name'] == 'Special Guard' or card_player2['name'] == 'Special Guard':
            pass
        # No caso de duas defesas
        elif conflict_result['winning_class'] == 'none':
            pass
        # No caso de uma defesa ganhar
        elif conflict_result['winning_class'] == 'guard':
            # Para uma defesa ganhar, a outra carta necessariamente é um ataque.
            # No caso da carta de defesa for a carta com efeito StopHittingYourself
            if card_player1['name'] == 'Iron Guard':
                reflected_damage = 0.20 * p1_base_defense
                self.player2.initial_HP -= reflected_damage
                self.gamejudge.log_message(f"Player 1 - {p1_name} - used Iron Guard! Reflected {reflected_damage} points of damage!")
                self.gamejudge.player_down(self.player1, self.player2)

            elif card_player2['name'] == 'Iron Guard':
                reflected_damage = 0.20 * p2_base_defense
                self.player1.initial_HP -= reflected_damage
                self.gamejudge.log_message(f"Player 2 - {p2_name} - used Iron Guard! Reflected {reflected_damage} points of damage!")
                self.gamejudge.player_down(self.player2, self.player1)

            else:
                pass

        # No caso de um clinch ganhar
        elif conflict_result['winning_class'] == 'clinch':
            # Se ambos usarem clinch
            if p1_current_action == 'clinch' and p2_current_action == 'clinch':
                self.gamejudge.set_draw_lock(self.player1)
                self.gamejudge.set_draw_lock(self.player2)
                self.gamejudge.log_message("Both players clinched! No drawing cards next turn")

            # Se somente o p1 usar clinch
            elif p1_current_action == 'clinch' and not p2_current_action == 'clinch':
                self.gamejudge.set_draw_lock(self.player2)

            # Somente o p2 usar clinch
            elif p2_current_action == 'clinch' and not p1_current_action == 'clinch':
                self.gamejudge.set_draw_lock(self.player1)
            
        # No caso de um ataque ganhar
        elif conflict_result['winning_class'] == 'attack':
            # Ataque contra clinch
            if p1_current_action == 'attack' and p2_current_action == 'clinch':
                # Dano base
                base_damage = self.damagecalculator.get_base_damage(self.player1, p2_weaknesses_multiplier, p2_resistances_multiplier)
                if card_player1['name'] == 'Strong Attack':
                    base_damage = 2 * base_damage
                    self.gamejudge.log_message(f"Player 1 - {p1_name} - used Strong Attack! Damage doubled to {base_damage}")
                elif card_player1['name'] == 'Rubber Attack':
                    self.gamejudge.set_draw_lock(self.player2)
                    self.gamejudge.log_message(f"Player 1 - {p1_name} - used Rubber Arm! The attack connected and now Player 2 - {p2_name} is clinched!")
                elif card_player1['name'] == 'Fiery Punch':
                    self.gamejudge.drop_hand_slot(self.player2, 1)
                    self.gamejudge.log_message(f"Player 1 - {p1_name} - used Fiery Punch! Player 2 - {p2_name} lost 1 hand slot!")

                
                # Aumenta o score
                self.gamejudge.give_point(self.player1)

                # Aplica o dano
                self.player2.initial_HP -= base_damage
                self.gamejudge.log_message(f"Player 1 - {p1_name} - Attack connected! {base_damage} points of damage applied!")
                self.gamejudge.player_down(self.player1, self.player2)

            elif p2_current_action == 'attack' and p1_current_action == 'clinch':
                # Dano base
                base_damage = self.damagecalculator.get_base_damage(self.player2, p1_weaknesses_multiplier, p1_resistances_multiplier)
                if card_player2['name'] == 'Strong Attack':
                    base_damage = 2 * base_damage
                    self.gamejudge.log_message(f"Player 2 - {p2_name} - used Strong Attack! Damage doubled to {base_damage}")
                elif card_player2['name'] == 'Rubber Attack':
                    self.gamejudge.set_draw_lock(self.player2)
                    self.gamejudge.log_message(f"Player 2 - {p2_name} - used Rubber Arm! The attack connected and now Player 1 - {p1_name} is clinched!")
                elif card_player2['name'] == 'Fiery Punch':
                    self.gamejudge.drop_hand_slot(self.player2, 1)
                    self.gamejudge.log_message(f"Player 2 - {p2_name} - used Fiery Punch! Player 1 - {p1_name} lost 1 hand slot!")

                
                # Aumenta o score
                self.gamejudge.give_point(self.player2)

                # Aplica o dano
                self.player1.initial_HP -= base_damage
                self.gamejudge.log_message(f"Player 2 - {p2_name} - Attack connected! {base_damage} points of damage applied!")
                self.gamejudge.player_down(self.player1, self.player2)

            # Dois ataques simultâneos
            elif p1_current_action == 'attack' and p2_current_action == 'attack':
                # Player 1 é mais ágil
                if self.player1.initial_agility > self.player2.initial_agility:

                    self._apply_attack(self.player1, self.player2, card_player1, p2_weaknesses_multiplier, p2_resistances_multiplier)

                    # Se o Player 2 estiver ainda apto a atacar
                    if not self.gamejudge.is_knocked_out(self.player2):
                        self._apply_attack(self.player2, self.player1, card_player2, p1_weaknesses_multiplier, p1_resistances_multiplier)

                # Player 2 é mais ágil                
                elif self.player2.initial_agility > self.player1.initial_agility:
                
                    self._apply_attack(self.player2, self.player1, card_player2, p1_weaknesses_multiplier, p1_resistances_multiplier)
                    
                    # Se o Player 1 estiver ainda apto a atacar
                    if not self.gamejudge.is_knocked_out(self.player2):
                        self._apply_attack(self.player1, self.player2, card_player1, p2_weaknesses_multiplier, p2_resistances_multiplier)

                # Os dois player tem a mesma agilidade
                elif self.player1.initial_agility == self.player2.initial_agility:
                    # Define a iniciativa aleatoriamente

                    if random.choice([True, False]):
                        first_attacker = self.player1
                        second_attacker = self.player2
                        card_first = card_player1
                        card_second = card_player2
                        weaknesses_first = p1_weaknesses_multiplier
                        resistances_first = p1_resistances_multiplier
                        weaknesses_second = p2_weaknesses_multiplier
                        resistances_second = p2_resistances_multiplier
                        
                        # Log de desempate
                        self.gamejudge.log_message(f"Agility tie! But Player 1 {self.player1.name} was a little faster!")
                    else:
                        first_attacker = self.player2
                        second_attacker = self.player1
                        card_first = card_player2
                        card_second = card_player1
                        weaknesses_first = p2_weaknesses_multiplier
                        resistances_first = p2_resistances_multiplier
                        weaknesses_second = p1_weaknesses_multiplier
                        resistances_second = p1_resistances_multiplier
                        
                        # Log de desempate
                        self.gamejudge.log_message(f"Agility tie! But Player 2 {self.player2.name} was a little faster!")


                    # Primeiro Ataque
                    self._apply_attack(first_attacker, second_attacker, card_first, weaknesses_second, resistances_second)
                    
                    # Checa se o alvo do primeiro ataque ainda está apto para revidar
                    if not self.gamejudge.is_knocked_out(second_attacker):
                        # Segundo Ataque (Revide)
                        self._apply_attack(second_attacker, first_attacker, card_second, weaknesses_first, resistances_first)

    def _apply_attack(self, attacker, target, card_attacker, target_weaknesses_multiplier, target_resistances_multiplier):
        
        # Calcular Dano Base
        base_damage = self.damagecalculator.get_base_damage(attacker, target_weaknesses_multiplier, target_resistances_multiplier)

        # Aplicar Efeitos da Carta do Atacante
        if card_attacker['name'] == 'Strong Attack':
            base_damage = 2 * base_damage
            self.gamejudge.log_message(f"Player {attacker.name} - used Strong Attack! Damage doubled to {base_damage}")
        
        elif card_attacker['name'] == 'Rubber Attack':
            self.gamejudge.set_draw_lock(target)
            self.gamejudge.log_message(f"Player {attacker.name} - used Rubber Arm! The attack connected and now Player {target.name} is clinched!")
        
        elif card_attacker['name'] == 'Fiery Punch':
            self.gamejudge.drop_hand_slot(target, 1)
            self.gamejudge.log_message(f"Player {attacker.name} - used Fiery Punch! Player {target.name} lost 1 hand slot!")
        
        # Aumenta o score
        self.gamejudge.give_point(attacker)

        # Aplica o dano no HP do alvo
        target.initial_HP -= base_damage
        self.gamejudge.log_message(f"Player {attacker.name} - Attack connected! {base_damage} points of damage applied!")

        # Checa se o alvo está nocauteado
        self.gamejudge.player_down(attacker, target)
        
        # Retorna o dano aplicado, caso seja necessário para a checagem do segundo ataque
        return base_damage                

# VAI TOMAR NO C* PQP VAMO TESTAR ISSO CAR*LH****************************************

In [53]:
# Criando a UI

In [54]:
ui = UIManager()

In [55]:
# Criando um jogador
ferpas = Player('Ferna', 'Terminator', 'atk', ui)

In [56]:
# Equipando itens no robo

In [57]:
iron_head = parts_list[0]
blazing_arm = parts_list[1]
rubber_arm = parts_list[2]
iron_body = parts_list[3]
rubber_body = parts_list[4]
iron_leg = parts_list[5]
rubber_leg = parts_list[6]

In [58]:
ferpas.robot.setSlot('head', iron_head)

Slot 'head' is Empty.

Slot head empty and available. Printing current robot stats.

                        Robot Terminator's Stats                        
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 6  │ 10  │ 8   │ 6   │ 12.0 │ 14.8 │ 14.0 │ N/A         │ N/A        │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴─────────────┴────────────┘

Part Iron Head equipped in slot head

New stats after part was equipped:

                         Robot Terminator's Stats                          
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 8  │ 10  │ 6   │ 8   │ 12.0 │ 14.8 │ 14.0 │ water, neutral │ fire       │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────┴────────────┘

            Robot Terminator Slots             
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Slot      ┃ Part ID ┃ Part Name ┃ Part Type ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ head      │ 1       │ Iron Head │ iron      │
│ body      │ N/A     │ N/A       │ N/A       │
│ right_arm │ N/A     │ N/A       │ N/A       │
│ left_arm  │ N/A     │ N/A       │ N/A       │
│ right_leg │ N/A     │ N/A       │ N/A       │
│ left_leg  │ N/A     │ N/A       │ N/A       │
└───────────┴─────────┴───────────┴───────────┘

In [59]:
ferpas.robot.setSlot('right_arm', blazing_arm)

Slot 'right_arm' is Empty.

Slot right_arm empty and available. Printing current robot stats.

                         Robot Terminator's Stats                          
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 8  │ 10  │ 6   │ 8   │ 12.0 │ 14.8 │ 14.0 │ water, neutral │ fire       │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────┴────────────┘

Part Blazing Arm equipped in slot right_arm

New stats after part was equipped:

                          Robot Terminator's Stats                          
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances    ┃ Weaknesses  ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 6  │ 12  │ 8   │ 6   │ 12.0 │ 14.8 │ 14.0 │ water, neutral │ fire, water │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────┴─────────────┘

             Robot Terminator Slots              
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Slot      ┃ Part ID ┃ Part Name   ┃ Part Type ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ head      │ 1       │ Iron Head   │ iron      │
│ body      │ N/A     │ N/A         │ N/A       │
│ right_arm │ 2       │ Blazing Arm │ fire      │
│ left_arm  │ N/A     │ N/A         │ N/A       │
│ right_leg │ N/A     │ N/A         │ N/A       │
│ left_leg  │ N/A     │ N/A         │ N/A       │
└───────────┴─────────┴─────────────┴───────────┘

In [60]:
paulo = Player('Colan', 'Mach35', 'bal', ui)

In [61]:
paulo.robot.setSlot('head', iron_head)

Slot 'head' is Empty.

Slot head empty and available. Printing current robot stats.

                          Robot Mach35's Stats                          
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 7  │ 7   │ 7   │ 7   │ 11.2 │ 11.2 │ 11.2 │ N/A         │ N/A        │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴─────────────┴────────────┘

Part Iron Head equipped in slot head

New stats after part was equipped:

                           Robot Mach35's Stats                            
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 9  │ 7   │ 5   │ 9   │ 11.2 │ 11.2 │ 11.2 │ water, neutral │ fire       │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────┴────────────┘

              Robot Mach35 Slots               
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Slot      ┃ Part ID ┃ Part Name ┃ Part Type ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ head      │ 1       │ Iron Head │ iron      │
│ body      │ N/A     │ N/A       │ N/A       │
│ right_arm │ N/A     │ N/A       │ N/A       │
│ left_arm  │ N/A     │ N/A       │ N/A       │
│ right_leg │ N/A     │ N/A       │ N/A       │
│ left_leg  │ N/A     │ N/A       │ N/A       │
└───────────┴─────────┴───────────┴───────────┘

In [62]:
paulo.robot.setSlot('body', iron_body)

Slot 'body' is Empty.

Slot body empty and available. Printing current robot stats.

                           Robot Mach35's Stats                            
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 9  │ 7   │ 5   │ 9   │ 11.2 │ 11.2 │ 11.2 │ water, neutral │ fire       │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────┴────────────┘

Part Iron Body equipped in slot body

New stats after part was equipped:

                                   Robot Mach35's Stats                                    
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances                    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 15 │ 7   │ -1  │ 13  │ 11.2 │ 11.2 │ 11.2 │ water, neutral, water, neutral │ fire, fire │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────────────────────┴────────────┘

              Robot Mach35 Slots               
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Slot      ┃ Part ID ┃ Part Name ┃ Part Type ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ head      │ 1       │ Iron Head │ iron      │
│ body      │ 4       │ Iron Body │ iron      │
│ right_arm │ N/A     │ N/A       │ N/A       │
│ left_arm  │ N/A     │ N/A       │ N/A       │
│ right_leg │ N/A     │ N/A       │ N/A       │
│ left_leg  │ N/A     │ N/A       │ N/A       │
└───────────┴─────────┴───────────┴───────────┘

In [63]:
paulo.robot.setSlot('left_arm', rubber_arm)

Slot 'left_arm' is Empty.

Slot left_arm empty and available. Printing current robot stats.

                                   Robot Mach35's Stats                                    
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances                    ┃ Weaknesses ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 15 │ 7   │ -1  │ 13  │ 11.2 │ 11.2 │ 11.2 │ water, neutral, water, neutral │ fire, fire │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴────────────────────────────────┴────────────┘

Part Rubber Arm equipped in slot left_arm

New stats after part was equipped:

                                         Robot Mach35's Stats                                          
┏━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ HP ┃ Con ┃ Str ┃ Agi ┃ Def  ┃ Atk  ┃ Cli  ┃ Resistances                          ┃ Weaknesses       ┃
┡━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 16 │ 6   │ 2   │ 13  │ 11.2 │ 11.2 │ 11.2 │ water, neutral, water, neutral, iron │ fire, fire, fire │
└────┴─────┴─────┴─────┴──────┴──────┴──────┴──────────────────────────────────────┴──────────────────┘

               Robot Mach35 Slots               
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Slot      ┃ Part ID ┃ Part Name  ┃ Part Type ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ head      │ 1       │ Iron Head  │ iron      │
│ body      │ 4       │ Iron Body  │ iron      │
│ right_arm │ N/A     │ N/A        │ N/A       │
│ left_arm  │ 3       │ Rubber Arm │ rubber    │
│ right_leg │ N/A     │ N/A        │ N/A       │
│ left_leg  │ N/A     │ N/A        │ N/A       │
└───────────┴─────────┴────────────┴───────────┘

In [64]:
# Robos equipados, criando o juiz e a calculadora
calculadora = DamageCalculator(ui)
juiz = GameJudge(ui, calculadora)

In [65]:
# Preparando os jogadores com as cartas especiais
ferpas.setDeck()
paulo.setDeck()

                                                Player Ferna's Deck                                                
┏━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ Card Name     ┃ Class  ┃ Type    ┃ Description                                                             ┃
┡━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 4  │ Strong Attack │ attack │ neutral │ Strong Attack. If the opponent clinches, it does double damage. Does    │
│    │               │        │         │ regular damage against Simple Guard.                                    │
│ 4  │ Strong Attack │ attack │ neutral │ Strong Attack. If the opponent clinches, it does double damage. Does    │
│    │               │        │         │ regular damage against Simple Guard.                                    │
│ 6  │ Iron Guard    │ guard  │ iron    │ The Iron Guard is so strong that it returns some damage to the oponent  │
│    │               │        │         │ if they choose to attack.                                               │
│ 7  │ Fiery Punch   │ attack │ fire    │ Attack that leaves the opponent with a burn. If it lands, the opponent  │
│    │               │        │         │ loses one slot of his hand for the end of the round.                    │
└────┴───────────────┴────────┴─────────┴─────────────────────────────────────────────────────────────────────────┘

                                                Player Colan's Deck                                                
┏━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ Card Name     ┃ Class  ┃ Type    ┃ Description                                                             ┃
┡━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 1  │ Simple Guard  │ guard  │ neutral │ Simple Guard. Protects against Attack from opponent.It still can be     │
│    │               │        │         │ Clinched                                                                │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 3  │ Clinch        │ clinch │ neutral │ Clinch, holds the opponent fighter. Vulnerable to opponents attacks.    │
│    │               │        │         │ Strong against opponents guards.                                        │
│ 4  │ Strong Attack │ attack │ neutral │ Strong Attack. If the opponent clinches, it does double damage. Does    │
│    │               │        │         │ regular damage against Simple Guard.                                    │
│ 5  │ Special Guard │ guard  │ neutral │ Special Guard. Blocks completely any attack. Invulnerable to clinches.  │
│ 6  │ Iron Guard    │ guard  │ iron    │ The Iron Guard is so strong that it returns some damage to the oponent  │
│    │               │        │         │ if they choose to attack.                                               │
│ 8  │ Rubber Attack │ attack │ rubber  │ Hits weakly and automatically clinches if it lands.                     │
└────┴───────────────┴────────┴─────────┴─────────────────────────────────────────────────────────────────────────┘

[{'id': 2,
  'category': 'card',
  'name': 'Simple Attack',
  'class': 'attack',
  'type': 'neutral',
  'description': 'Simple Attack. Does damage even if the opponent Attacks or Clinches. It can be blocked by Simple Guard.',
  'effect': None},
 {'id': 2,
  'category': 'card',
  'name': 'Simple Attack',
  'class': 'attack',
  'type': 'neutral',
  'description': 'Simple Attack. Does damage even if the opponent Attacks or Clinches. It can be blocked by Simple Guard.',
  'effect': None},
 {'id': 2,
  'category': 'card',
  'name': 'Simple Attack',
  'class': 'attack',
  'type': 'neutral',
  'description': 'Simple Attack. Does damage even if the opponent Attacks or Clinches. It can be blocked by Simple Guard.',
  'effect': None},
 {'id': 1,
  'category': 'card',
  'name': 'Simple Guard',
  'class': 'guard',
  'type': 'neutral',
  'description': 'Simple Guard. Protects against Attack from opponent.It still can be Clinched',
  'effect': None},
 {'id': 1,
  'category': 'card',
  'name': 'Simpl

In [66]:
turno = Turn(ferpas, paulo, juiz, calculadora)

In [67]:
turno.execute_first_phase()

Turn 0 - Judge: Ferna invulnerability disabled!

Turn 0 - Judge: Colan invulnerability disabled!

In [69]:
turno.execute_second_phase()

                                                Player Ferna's Hand                                                
┏━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ Card Name     ┃ Class  ┃ Type    ┃ Description                                                             ┃
┡━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
└────┴───────────────┴────────┴─────────┴─────────────────────────────────────────────────────────────────────────┘

Choose a card: 0


                                                Player Colan's Hand                                                
┏━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃ Card Name     ┃ Class  ┃ Type    ┃ Description                                                             ┃
┡━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2  │ Simple Attack │ attack │ neutral │ Simple Attack. Does damage even if the opponent Attacks or Clinches. It │
│    │               │        │         │ can be blocked by Simple Guard.                                         │
└────┴───────────────┴────────┴─────────┴─────────────────────────────────────────────────────────────────────────┘

Choose a card: 0


Turn 0 - Judge: Ferna played 'Simple Attack'.

Turn 0 - Judge: Colan played 'Simple Attack'.

In [70]:
turno.execute_third_phase()

Turn 0 - Player Ferna - Attack connected! 7.4 points of damage applied!

Turn 0 - Player Colan discarded 2 cards from deck and 0 cards from hand.

Turn 0 - Player Colan discarded 2 cards from deck and 0 cards from hand.

In [71]:
ferpas.initial_HP

6

In [72]:
paulo.initial_HP

-0.40000000000000036

In [73]:
turno.execute_first_phase()

Turn 0 - Judge: Ferna invulnerability disabled!

Turn 0 - Judge: Colan invulnerability disabled!